<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=196f87dd769084b72ffa729a3b49a73e67c4af2326cda9f0cfc73c0bab2d65c2
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 15:26:21
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: -86.85 K (-0.62%)
Current PnL: -26.86 L (-17.35%)
CY Booked + Current PnL: -12.57 L (-8.12%)
-------------------
Total profit:  1.38 L
Total loss:  -28.24 L
-------------------
Total Booked + Current PnL: 14.41 L (11.27%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.19%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.49 L (65.26%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.91,-18.04,22.13,0.09,18278.0,-18181.0,82596.0,91.68,35.0,M-SC,2.33,253.0,-0.99,0.59,8.51,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.05,8.60,10.21,19.69,21260.0,16488.0,208230.0,-2.04,57.0,X-MC,2.52,78.0,0.78,1.49,15.98,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-2.09,-13.52,15.66,0.02,23640.0,-23601.0,150958.0,130.47,49.0,M-SC,14.77,234.0,-1.00,1.08,31.31,OX40N,NTT,PAINTS
50,MASFIN,398.61,-2.63,-2.66,25.39,22.05,24214.0,-2610.0,95370.0,-15.99,53.0,H-SC,7.22,164.0,-0.11,0.68,38.63,XR,ATH,FINANCE
43,ITC,452.00,-0.12,-1.52,12.49,10.78,29559.0,-3658.0,236660.0,-41.15,45.0,X-LC,1.95,5.0,-0.12,1.70,3.63,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CERA,9475.00,3.42,-29.96,76.89,23.89,94728.0,-52703.0,123200.0,-32.46,41.0,H-SC,10.56,157.0,-0.56,0.88,6.40,OX40N,NTT,CERAMICS
25,FINCABLES,1641.55,2.72,-1.35,110.77,107.91,161331.0,-1999.0,145645.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.60,OX40N,ATH,CABLES
6,ASIANTILES,137.00,2.15,-16.73,118.19,81.67,92319.0,-15699.0,78111.0,7016.67,46.0,L-SC,12.37,271.0,-0.17,0.56,51.26,XR,NTT,CERAMICS
44,JCHAC,2282.00,1.70,-33.84,51.13,-0.01,43236.0,-43245.0,84560.0,17204.35,39.0,M-SC,7.32,233.0,-1.00,0.61,8.23,OX40N,NTT,AC
71,TANLA,1943.92,0.97,-35.81,242.24,119.69,421032.0,-96953.0,173808.0,-35.11,47.0,H-SC,8.00,156.0,-0.23,1.25,37.31,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-3.26,-40.31,126.63,35.27,85825.0,-45773.0,67776.0,-53.72,48.0,L-SC,10.72,270.0,-0.53,0.49,65.88,XR,NTT,HOTELS
57,RAJOOENG,143.10,-2.85,-35.71,117.15,39.61,77202.0,-36600.0,65900.0,-56.61,34.0,H-SC,28.73,135.0,-0.47,0.47,3.83,AR,ATH,MISC
50,MASFIN,398.61,-2.63,-2.66,25.39,22.05,24214.0,-2610.0,95370.0,-15.99,53.0,H-SC,7.22,164.0,-0.11,0.68,38.63,XR,ATH,FINANCE
0,5PAISA,593.00,-2.57,-41.19,91.97,12.90,118183.0,-89994.0,128502.0,89.90,44.0,H-SC,12.03,161.0,-0.76,0.92,6.85,OX40N,NTT,FINANCE
38,INDIAMART,4810.62,-2.32,-4.86,117.28,106.72,137619.0,-5994.0,117342.0,-52.92,33.0,H-SC,3.02,139.0,-0.04,0.84,17.38,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.72,-1.35,110.77,107.91,161331.0,-1999.0,145645.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.60,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.09,-13.52,15.66,0.02,23640.0,-23601.0,150958.0,130.47,49.0,M-SC,14.77,234.0,-1.00,1.08,31.31,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.91,-18.04,22.13,0.09,18278.0,-18181.0,82596.0,91.68,35.0,M-SC,2.33,253.0,-0.99,0.59,8.51,OX40N,NTT,DURABLES
66,SIS,528.00,0.20,-24.07,59.71,21.26,50337.0,-26729.0,84303.0,1989.12,50.0,H-SC,5.11,166.0,-0.53,0.60,13.90,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.70,-24.98,51.25,13.47,103684.0,-67356.0,202311.0,-69.09,35.0,H-SC,5.20,158.0,-0.65,1.45,2.87,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.03,6.25,62.14,72.27,111873.0,10585.0,180033.0,-8.8,63.0,M-LC,2.45,99.0,0.09,1.29,12.81,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.03,6.25,62.14,72.27,111873.0,10585.0,180033.0,-8.80,63.0,M-LC,2.45,99.0,0.09,1.29,12.81,XR,NTT,IT
25,FINCABLES,1641.55,2.72,-1.35,110.77,107.91,161331.0,-1999.0,145645.0,-19.74,60.0,M-SC,12.52,216.0,-0.01,1.04,8.60,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-2.32,-4.86,117.28,106.72,137619.0,-5994.0,117342.0,-52.92,33.0,H-SC,3.02,139.0,-0.04,0.84,17.38,AR,ATH,MISC
1,ABB,7934.00,-1.05,-4.05,51.72,45.57,129825.0,-10603.0,251016.0,-38.36,55.0,H-MC,4.56,120.0,-0.08,1.80,6.81,AR,NTT,ELECTRICAL
37,IEX,219.00,-1.57,-4.93,56.33,48.63,107084.0,-9852.0,190102.0,-36.71,46.0,H-SC,14.07,136.0,-0.09,1.36,6.06,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.32,-18.21,90.09,55.47,179261.0,-44306.0,198980.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.43,1.95,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-1.21,-9.90,42.18,28.11,76546.0,-19935.0,181475.0,-34.75,27.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.52,-29.22,73.74,22.98,117908.0,-66001.0,159897.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.15,0.57,XY24,NTT,AUTO
8,AWL,485.00,-1.68,-23.35,100.25,53.50,231879.0,-70450.0,231301.0,-62.86,30.0,X-MC,2.81,58.0,-0.30,1.66,2.74,XY24,NTT,FMCG
33,HINDUNILVR,2922.00,-0.65,-11.36,28.24,13.67,64348.0,-29203.0,227860.0,-17.95,31.0,X-LC,2.15,14.0,-0.45,1.63,7.57,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.62,-1.66,26.58,24.48,65829.0,-4191.0,247665.0,-81.32,43.0,X-SC,0.30,86.0,-0.06,1.78,21.09,X40N,NTT,MISC
43,ITC,452.0,-0.12,-1.52,12.49,10.78,29559.0,-3658.0,236660.0,-41.15,45.0,X-LC,1.95,5.0,-0.12,1.70,3.63,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.65,-11.36,28.24,13.67,64348.0,-29203.0,227860.0,-17.95,31.0,X-LC,2.15,14.0,-0.45,1.63,7.57,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.89,5.29,24.17,30.73,50133.0,10413.0,207418.0,-19.31,57.0,X-MC,2.22,75.0,0.21,1.49,18.73,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.60,6.59,13.30,20.77,33193.0,15428.0,249572.0,-2.63,58.0,X-MC,2.46,62.0,0.46,1.79,13.96,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.40,-25.59,120.62,64.15,213558.0,-60901.0,177050.0,-56.31,34.0,X-SC,2.60,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.28,-38.91,121.33,35.20,95367.0,-50069.0,78601.0,3.10,31.0,X-SC,16.13,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,-1.21,-9.90,42.18,28.11,76546.0,-19935.0,181475.0,-34.75,27.0,X-MC,12.94,55.0,-0.26,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.52,-29.22,73.74,22.98,117908.0,-66001.0,159897.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.15,0.57,XY24,NTT,AUTO
59,RELAXO,1176.00,-0.90,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,40.0,X-SC,6.95,91.0,-0.48,0.54,0.81,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.59,-29.41,102.35,42.84,46967.0,-19117.0,45889.0,-53.72,43.0,X-SC,35.52,83.0,-0.41,0.33,3.82,XY24,NTT,MISC
59,RELAXO,1176.00,-0.90,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,40.0,X-SC,6.95,91.0,-0.48,0.54,0.81,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.28,-38.91,121.33,35.20,95367.0,-50069.0,78601.0,3.10,31.0,X-SC,16.13,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.71,-4.81,30.35,24.08,36675.0,-6110.0,120840.0,-12.55,32.0,X-SC,5.12,89.0,-0.17,0.87,13.86,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.48,-16.74,72.15,43.32,97835.0,-27272.0,135600.0,-28.43,36.0,X-SC,4.76,90.0,-0.28,0.97,4.12,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.75,-11.85,36.93,20.71,123133.0,-44813.0,333424.0,-23.95,59.0,X-LC,4.46,1.0,-0.36,2.39,11.41,X40,ATH,IT
41,INFY,1972.00,0.54,9.38,22.73,34.24,77793.0,29360.0,342248.0,-14.93,64.0,X-LC,3.59,2.0,0.38,2.45,18.44,X40,NTT,IT
75,TMPV,600.00,-0.52,-29.22,73.74,22.98,117908.0,-66001.0,159897.0,-26.74,29.0,X-LC,6.80,3.0,-0.56,1.15,0.57,XY24,NTT,AUTO
81,VBL,671.64,-0.37,-3.88,41.15,35.67,124928.0,-12250.0,303592.0,-15.30,55.0,X-LC,4.13,4.0,-0.10,2.18,9.44,X40N,ATH,FMCG
43,ITC,452.00,-0.12,-1.52,12.49,10.78,29559.0,-3658.0,236660.0,-41.15,45.0,X-LC,1.95,5.0,-0.12,1.70,3.63,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.73,-37.61,113.01,32.89,54435.0,-29038.0,48168.0,-695.06,54.0,L-MC,5.97,259.0,-0.53,0.35,32.65,XR,NTT,BANKS
6,ASIANTILES,137.00,2.15,-16.73,118.19,81.67,92319.0,-15699.0,78111.0,7016.67,46.0,L-SC,12.37,271.0,-0.17,0.56,51.26,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.63,-2.66,25.39,22.05,24214.0,-2610.0,95370.0,-15.99,53.0,H-SC,7.22,164.0,-0.11,0.68,38.63,XR,ATH,FINANCE
13,BSOFT,831.70,-2.10,-22.84,95.42,50.79,103557.0,-32125.0,108528.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.36,XR,ATH,IT
39,INDIGOPNTS,1408.00,-2.09,-13.52,15.66,0.02,23640.0,-23601.0,150958.0,130.47,49.0,M-SC,14.77,234.0,-1.00,1.08,31.31,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-2.10,-22.84,95.42,50.79,103557.0,-32125.0,108528.0,-1.04,62.0,H-SC,6.32,151.0,-0.31,0.78,26.36,XR,ATH,IT
84,WIPRO,420.0,-1.03,6.25,62.14,72.27,111873.0,10585.0,180033.0,-8.80,63.0,M-LC,2.45,99.0,0.09,1.29,12.81,XR,NTT,IT
41,INFY,1972.0,0.54,9.38,22.73,34.24,77793.0,29360.0,342248.0,-14.93,64.0,X-LC,3.59,2.0,0.38,2.45,18.44,X40,NTT,IT
1,ABB,7934.0,-1.05,-4.05,51.72,45.57,129825.0,-10603.0,251016.0,-38.36,55.0,H-MC,4.56,120.0,-0.08,1.80,6.81,AR,NTT,ELECTRICAL
27,GLAXO,3466.2,0.21,3.71,34.17,39.15,70620.0,7388.0,206672.0,-16.74,53.0,X-MC,10.70,60.0,0.10,1.48,33.78,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.89
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.92
MC    31.00
LC    24.11
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      23.00
X40N     13.10
XR        9.75
XY25      9.08
AR        9.05
OX40N     7.58
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.51
H-SC    23.76
X-LC    20.68
M-SC    11.69
X-SC     8.05
H-MC     4.82
L-SC     1.42
M-MC     1.32
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.40,-7.53,42.11
IT,13.03,-15.67,73.83
FINANCE,9.46,-20.67,73.15
MISC,7.25,-28.41,80.93
ELECTRICAL,6.02,-11.66,52.56
PAINTS,5.85,-13.10,29.90
INSURANCE,4.77,-2.06,36.99
PHARMA,4.13,-2.15,34.30
AUTO,2.82,-33.45,79.72


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3197696.0,21
AR,1309902.0,10
XR,1301859.0,13
X40,1014114.0,14
X40N,860447.0,9
OX40N,732545.0,10
XY25,373702.0,6
SR,282116.0,2
MH,76989.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3661217.0,25
M-SC,1427310.0,15
X-MC,1410140.0,16
X-LC,842018.0,11
X-SC,796595.0,8
H-MC,407482.0,3
L-SC,263753.0,3
M-LC,111873.0,1
H-LC,75488.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1280011.0      6
           AR         895898.0      5
M-SC       XY24       823029.0      6
H-SC       XR         783669.0      7
X-MC       X40        492505.0      7
           XY24       405200.0      3
H-SC       OX40N      342534.0      4
X-MC       X40N       334203.0      4
X-LC       X40        331654.0      5
X-SC       X40N       309440.0      3
M-SC       OX40N      304402.0      5
X-SC       XY24       297200.0      3
H-SC       SR         282116.0      2
X-LC       X40N       216804.0      2
H-MC       AR         212375.0      2
X-LC       XY24       197149.0      2
H-MC       XY24       195107.0      1
X-SC       X40        189955.0      2
X-MC       XY25       178232.0      2
L-SC       XR         178144.0      2
M-SC       XR         173738.0      2
           AR         126141.0      2
M-LC       XR         111873.0      1
X-LC       XY25        96411.0      2
L-SC       OX40N       85609.0      1
H-SC       MH          76989.0      1
H-LC       AR          75488.0      1
M-MC       XY25        60922.0      1
L-MC       XR          54435.0      1
L-LC       XY25        38137.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
